In [1]:
import streamlit as st
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input

# The file extensions for the preprocessed data and trained models should be as follows:

# Preprocessed Data:

# The cnn_data and gradcam_data variables are loaded from .npy files, which are Numpy array files. These files should have the .npy extension.
# Trained Models:

# The cnn_model and gradcam_model variables are loaded from saved models using the load_model function from TensorFlow. These models should have the .h5 extension, which is a common extension for saving Keras models.

# Load the preprocessed data
cnn_data = np.load('path_to_cnn_data.npy')
gradcam_data = np.load('path_to_gradcam_data.npy')

# Load the trained CNN model
cnn_model = load_model('path_to_cnn_model.h5')

# Load the trained GradCAM model
gradcam_model = load_model('path_to_gradcam_model.h5')

def preprocess_image(image):
    # Resize the image to match the input size of the models
    image = cv2.resize(image, (224, 224))

    # Convert the image to RGB format
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Preprocess the image for the CNN model
    cnn_image = preprocess_input(img_to_array(image))

    # Preprocess the image for the GradCAM model
    gradcam_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert back to BGR format
    gradcam_image = preprocess_input(img_to_array(gradcam_image))

    return cnn_image, gradcam_image

def detect_autism(image):
    # Preprocess the image
    cnn_image, gradcam_image = preprocess_image(image)

    # Perform autism detection using the models
    cnn_result = cnn_model.predict(np.expand_dims(cnn_image, axis=0))[0]
    gradcam_result = gradcam_model.predict(np.expand_dims(gradcam_image, axis=0))[0]

    # Calculate the likelihood of being autistic or not
    autism_probability = cnn_result[1]
    non_autism_probability = cnn_result[0]
    total_probability = autism_probability + non_autism_probability

    # Determine which category the face matches more closely
    if autism_probability > non_autism_probability:
        result = "High chance of autism"
    else:
        result = "Low chance of autism"

    return result, autism_probability / total_probability

def main():
    st.set_page_config(
        page_title="Facial Assessment Tool",
        page_icon="🧩",
        layout="centered",
        initial_sidebar_state="collapsed",
    )

    st.title('Facial Assessment Tool')

    # Display the introduction and instructions
    st.markdown('''
        ## Facial Assessment Tool
        Upload an image of a child to assess the likelihood of autism based on facial morphology.
        ''')

    # Upload image
    uploaded_file = st.file_uploader('Upload an image', type=['jpg', 'jpeg', 'png'])

    if uploaded_file is not None:
        # Read image file
        image = np.array(bytearray(uploaded_file.read()), dtype=np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)

        # Perform autism detection
        result, probability = detect_autism(image)

        # Display the result
        st.subheader('Detection Result:')
        st.success(f'Likelihood of being autistic: {probability:.2f}')
        st.info(f'Classification: {result}')

        # Display the image
        st.image(image, caption='Uploaded Image', use_column_width=True)

    # Citation
    st.markdown('---')
    st.markdown('''
        ### Citation
        This tool is backed up by research papers conducted by Naomi Scott, Alex Lee Jones, Robin Stewart Samuel Kramer, Robert Ward, Mohammad-Parsa Hosseini, Madison Beary, Alex Hadsell, 
        Ryan Messersmith, Hamid Soltanian-Zadeh, K.K. Mujeeb Rahman, and M. Monica Subashini. You can find the studies at the following links:
        
        - [Bangor University Study](https://ward-lab.bangor.ac.uk/pubs/Scott_Ward_14_AQ.pdf)
        - [Deep Learning for Autism Diagnosis and Facial Analysis in Children](https://www.frontiersin.org/articles/10.3389/fncom.2021.789998/full)
        - [Identification of Autism in Children Using Static Facial Features and Deep Neural Networks](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8773918/)
        
        Please cite the appropriate studies when using this tool in your work.
        ''')

if __name__ == '__main__':
    main()

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates